In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os


Model Path

In [ ]:
MODEL_PATH = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/03_Models/bone_fracture_model_phase1.h5"


Model Load Karo

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)
print("✅ Model loaded successfully")


✅ Model loaded successfully


Image Preprocessing Function

In [ ]:
IMG_SIZE = (224, 224)

def preprocess_image(img_path):
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Image not found: {img_path}")

    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    return img_array


Prediction Logic

In [ ]:
CLASS_NAMES = ["No Fracture", "Fracture"]

def predict_fracture(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)[0][0]

    if prediction >= 0.5:
        result = "Fracture"
        confidence = prediction * 100
    else:
        result = "No Fracture"
        confidence = (1 - prediction) * 100

    return result, round(confidence, 2)


TEST

In [ ]:
test_image = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/01_Data/raw/bone fracture dataset/test/fractured/1-rotated1-rotated2-rotated1.jpg"

result, confidence = predict_fracture(test_image)

print("Prediction:", result)
print("Confidence:", confidence, "%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
Prediction: No Fracture
Confidence: 99.81 %


**PART-2 STRUCTURE**


Explanation Templates

In [ ]:
def generate_patient_explanation(result, confidence):
    if result == "Fracture":
        explanation = {
            "diagnosis": "Bone Fracture Detected",
            "summary": (
                "The X-ray image suggests a possible bone fracture. "
                "This means the continuity of the bone may be disrupted."
            ),
            "possible_causes": [
                "Fall from height",
                "Road traffic accident",
                "Direct impact or trauma",
                "Sports-related injury"
            ],
            "care_advice": [
                "Avoid putting weight or pressure on the affected area",
                "Immobilize the limb if possible",
                "Apply ice to reduce swelling",
                "Consult an orthopedic doctor immediately"
            ]
        }
    else:
        explanation = {
            "diagnosis": "No Fracture Detected",
            "summary": (
                "The X-ray image does not show clear signs of a bone fracture. "
                "However, soft tissue injury or minor cracks may still exist."
            ),
            "possible_causes": [
                "Muscle strain",
                "Ligament injury",
                "Minor impact trauma"
            ],
            "care_advice": [
                "Rest the affected area",
                "Apply ice if there is pain or swelling",
                "Monitor symptoms for 24–48 hours",
                "Consult a doctor if pain persists"
            ]
        }

    explanation["confidence"] = f"{confidence}%"
    explanation["disclaimer"] = (
        "⚠ This is an AI-assisted analysis and not a medical diagnosis. "
        "Please consult a qualified medical professional."
    )

    explanation["copyright"] = "© SHUBHAM MADDHESIYA"

    return explanation


Combine Prediction + Explanation

In [ ]:
def full_patient_report(img_path):
    result, confidence = predict_fracture(img_path)
    explanation = generate_patient_explanation(result, confidence)

    report = {
        "result": result,
        "confidence": confidence,
        "details": explanation
    }

    return report


TEST (Patient View Output)

In [ ]:
patient_report = full_patient_report(test_image)

for key, value in patient_report.items():
    print("\n", key.upper())
    print(value)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

 RESULT
No Fracture

 CONFIDENCE
99.81

 DETAILS
{'diagnosis': 'No Fracture Detected', 'summary': 'The X-ray image does not show clear signs of a bone fracture. However, soft tissue injury or minor cracks may still exist.', 'possible_causes': ['Muscle strain', 'Ligament injury', 'Minor impact trauma'], 'care_advice': ['Rest the affected area', 'Apply ice if there is pain or swelling', 'Monitor symptoms for 24–48 hours', 'Consult a doctor if pain persists'], 'confidence': '99.80999755859375%', 'disclaimer': '⚠ This is an AI-assisted analysis and not a medical diagnosis. Please consult a qualified medical professional.', 'copyright': '© SHUBHAM MADDHESIYA'}


**part 3 code,**

folder define

In [ ]:
import os

UPLOAD_DIR = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/04_Uploads"

os.makedirs(UPLOAD_DIR, exist_ok=True)

print("Upload folder ready:", UPLOAD_DIR)


Upload folder ready: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/04_Uploads


Image Validation

In [ ]:
from PIL import Image

ALLOWED_EXTENSIONS = (".png", ".jpg", ".jpeg")

def is_valid_image(file_path):
    if not file_path.lower().endswith(ALLOWED_EXTENSIONS):
        return False

    try:
        img = Image.open(file_path)
        img.verify()
        return True
    except Exception:
        return False


Simulated Patient Upload Function

In [ ]:
def patient_upload_image(source_image_path):
    if not os.path.exists(source_image_path):
        raise FileNotFoundError("❌ Image file not found")

    if not is_valid_image(source_image_path):
        raise ValueError("❌ Invalid or corrupted image")

    file_name = os.path.basename(source_image_path)
    destination_path = os.path.join(UPLOAD_DIR, file_name)

    with open(source_image_path, "rb") as src, open(destination_path, "wb") as dst:
        dst.write(src.read())

    return destination_path


Upload → Prediction → Explanation

In [ ]:
def patient_upload_and_analyze(image_path):
    uploaded_image = patient_upload_image(image_path)
    report = full_patient_report(uploaded_image)
    return report


FINAL TEST

In [ ]:
uploaded_report = patient_upload_and_analyze(test_image)

print("\n🩻 AI FRACTURE ANALYSIS REPORT")
print("-" * 40)

print("Result:", uploaded_report["result"])
print("Confidence:", uploaded_report["confidence"], "%")

details = uploaded_report["details"]

print("\nDiagnosis:", details["diagnosis"])
print("\nSummary:", details["summary"])

print("\nPossible Causes:")
for cause in details["possible_causes"]:
    print("-", cause)

print("\nCare Advice:")
for advice in details["care_advice"]:
    print("-", advice)

print("\n", details["disclaimer"])
print(details["copyright"])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step

🩻 AI FRACTURE ANALYSIS REPORT
----------------------------------------
Result: No Fracture
Confidence: 99.81 %

Diagnosis: No Fracture Detected

Summary: The X-ray image does not show clear signs of a bone fracture. However, soft tissue injury or minor cracks may still exist.

Possible Causes:
- Muscle strain
- Ligament injury
- Minor impact trauma

Care Advice:
- Rest the affected area
- Apply ice if there is pain or swelling
- Monitor symptoms for 24–48 hours
- Consult a doctor if pain persists

 ⚠ This is an AI-assisted analysis and not a medical diagnosis. Please consult a qualified medical professional.
© SHUBHAM MADDHESIYA


**STEP 4**

In [ ]:
PDF_DIR = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_PDF_Reports"
os.makedirs(PDF_DIR, exist_ok=True)

print("PDF report folder ready:", PDF_DIR)


PDF report folder ready: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_PDF_Reports


In [ ]:
!pip install reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.5 MB/s eta 0:00:00


PDF Generator Function

In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.lib.utils import ImageReader
from datetime import datetime


In [ ]:
def generate_pdf_report(image_path, report_data):
    file_name = f"Bone_Fracture_Report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pdf"
    pdf_path = os.path.join(PDF_DIR, file_name)

    c = canvas.Canvas(pdf_path, pagesize=A4)
    width, height = A4

    # ===== TITLE =====
    c.setFont("Helvetica-Bold", 18)
    c.drawCentredString(width / 2, height - 2 * cm, "AI Bone Fracture Analysis Report")

    # ===== IMAGE =====
    c.drawImage(
        ImageReader(image_path),
        2 * cm,
        height - 12 * cm,
        width=8 * cm,
        height=8 * cm,
        preserveAspectRatio=True
    )

    # ===== REPORT TEXT =====
    x = 11 * cm
    y = height - 4 * cm

    c.setFont("Helvetica-Bold", 12)
    c.drawString(x, y, f"Result: {report_data['result']}")
    y -= 1 * cm

    c.drawString(x, y, f"Confidence: {report_data['confidence']} %")
    y -= 1 * cm

    c.drawString(x, y, "Diagnosis:")
    y -= 0.7 * cm

    c.setFont("Helvetica", 11)
    c.drawString(x, y, report_data["details"]["diagnosis"])
    y -= 1.2 * cm

    c.setFont("Helvetica-Bold", 12)
    c.drawString(x, y, "Summary:")
    y -= 0.7 * cm

    c.setFont("Helvetica", 11)
    c.drawString(x, y, report_data["details"]["summary"])
    y -= 1.2 * cm

    c.setFont("Helvetica-Bold", 12)
    c.drawString(x, y, "Possible Causes:")
    y -= 0.7 * cm

    c.setFont("Helvetica", 11)
    for cause in report_data["details"]["possible_causes"]:
        c.drawString(x, y, f"- {cause}")
        y -= 0.6 * cm

    y -= 0.5 * cm
    c.setFont("Helvetica-Bold", 12)
    c.drawString(x, y, "Care Advice:")
    y -= 0.7 * cm

    c.setFont("Helvetica", 11)
    for advice in report_data["details"]["care_advice"]:
        c.drawString(x, y, f"- {advice}")
        y -= 0.6 * cm

    # ===== DISCLAIMER =====
    y -= 1 * cm
    c.setFont("Helvetica-Oblique", 9)
    c.drawString(2 * cm, y, report_data["details"]["disclaimer"])

    # ===== BRANDING =====
    y -= 0.8 * cm
    c.drawString(2 * cm, y, report_data["details"]["copyright"])

    c.save()
    return pdf_path


In [ ]:
pdf_path = generate_pdf_report(test_image, uploaded_report)
print("✅ PDF Report Generated:")
print(pdf_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_PDF_Reports/Bone_Fracture_Report_20260203_094920.pdf'

In [ ]:
import os

pdf_path = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_PDF_Reports"
print("Folder exists:", os.path.exists(pdf_path))
print("Files inside folder:")
print(os.listdir(pdf_path))


Folder exists: False
Files inside folder:


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bone_Fracture_Detection_CNN/05_PDF_Reports'

others..

In [ ]:
# ===== PROJECT ROOT =====
PROJECT_ROOT = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN"

# ===== FIXED PATHS =====
MODEL_PATH = f"{PROJECT_ROOT}/03_Models/bone_fracture_model_phase1.h5"
REPORTS_DIR = f"{PROJECT_ROOT}/06_Reports"

# safety
import os
os.makedirs(REPORTS_DIR, exist_ok=True)

print("✅ Model Path:", MODEL_PATH)
print("✅ Reports Path:", REPORTS_DIR)


✅ Model Path: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/03_Models/bone_fracture_model_phase1.h5
✅ Reports Path: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports


In [ ]:
import os
from datetime import datetime

# ===== PROJECT ROOT =====
PROJECT_ROOT = "/content/drive/MyDrive/Bone_Fracture_Detection_CNN"

# ===== FIXED FOLDERS =====
UPLOADS_DIR = os.path.join(PROJECT_ROOT, "04_Inference")
REPORTS_DIR = os.path.join(PROJECT_ROOT, "06_Reports")

# ===== ENSURE FOLDERS EXIST =====
os.makedirs(UPLOADS_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

print("✅ Uploads Dir:", UPLOADS_DIR)
print("✅ Reports Dir:", REPORTS_DIR)


✅ Uploads Dir: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/04_Inference
✅ Reports Dir: /content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports


In [ ]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm

def generate_pdf_report(image_path, prediction, confidence):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    pdf_path = os.path.join(
        REPORTS_DIR,
        f"Bone_Fracture_Report_{timestamp}.pdf"
    )

    c = canvas.Canvas(pdf_path, pagesize=A4)

    c.setFont("Helvetica-Bold", 16)
    c.drawString(2*cm, 28*cm, "Bone Fracture Detection Report")

    c.setFont("Helvetica", 11)
    c.drawString(2*cm, 26.5*cm, f"Prediction: {prediction}")
    c.drawString(2*cm, 25.8*cm, f"Confidence: {confidence:.2f}%")

    c.setFont("Helvetica-Oblique", 9)
    c.drawString(2*cm, 2*cm, "© SHUBHAM MADDHESIYA")

    c.showPage()
    c.save()

    print("✅ PDF GENERATED AT:")
    print(pdf_path)

    return pdf_path


In [ ]:
generate_pdf_report("dummy.jpg", "Fracture Detected", 94.6)


✅ PDF GENERATED AT:
/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports/Bone_Fracture_Report_20260203_095450.pdf


'/content/drive/MyDrive/Bone_Fracture_Detection_CNN/06_Reports/Bone_Fracture_Report_20260203_095450.pdf'